# CONFIG

In [178]:
import streamlit as st
import sqlalchemy as db
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import time

# DATA STRUCTURING

### Q1 DATA

In [179]:
#connect to the SQL database
connection = sqlite3.connect('bce.db')
#query tables
sql_query_Q1T1 = pd.read_sql_query ('''
                                SELECT count(EnterpriseNumber) as Total_Enterprise, Description
                                FROM enterprise
                                INNER JOIN code ON enterprise.JuridicalForm = code.Code
                                WHERE code.Language = 'FR' AND code.Category = 'JuridicalForm'
                                Group by Description
                                Order by count(EnterpriseNumber) DESC;
                                ''', connection)

#Create a dataframe 1 to prepare graph input
df_Q1T1 = pd.DataFrame(sql_query_Q1T1, columns = ['Total_Enterprise', 'Description'])

df_Q1T1_most_frequent = df_Q1T1.iloc[:6,:]
data = [{'Description':'Autres','Total_Enterprise':df_Q1T1['Total_Enterprise'][6:].sum()}]
data_df = pd.DataFrame(data)
df_Q1T1 = pd.concat([df_Q1T1_most_frequent, data_df])

### Q2 DATA

In [180]:
#query tables : Which percentage of the companies are under which Status?
sql_query_Q2 = pd.read_sql_query ('''
                               SELECT count(EnterpriseNumber), Status
                               FROM enterprise
                               ''', connection)

#Create dataframe
df_Q2 = pd.DataFrame(sql_query_Q2)
score_active_ent = df_Q2.iat[0,0]
print(str(score_active_ent))


1863292


### Q3 DATA

In [181]:
#query tables : Which percentage of the companies are which type of entreprise?
sql_query_Q3T1 = pd.read_sql_query ('''
                               SELECT count(EnterpriseNumber), TypeOfEnterprise
                               FROM enterprise
                               Group by TypeOfEnterprise
                               ''', connection)

sql_query_Q3T2 = pd.read_sql_query ('''
                               SELECT *
                               FROM code
                               WHERE Language=="FR" AND Category=="TypeOfEnterprise"
                               ''', connection)

df_Q3T1 = pd.DataFrame(sql_query_Q3T1)
df_Q3T1.rename(columns = {'TypeOfEnterprise':'Code'}, inplace = True)
df_Q3T2 = pd.DataFrame(sql_query_Q3T2)
df_Q3 = pd.merge(df_Q3T1, df_Q3T2, on='Code')
df_Q3 = df_Q3[["count(EnterpriseNumber)", "Description"]]
df_Q3


,count(EnterpriseNumber),Description
0,763643,Personne physique
1,1099649,Personne morale


### Q4 DATA

In [188]:
#ANALYSIS - Whether or not using NACE 2003?
sql_query_Q4 = pd.read_sql_query ('''
                               SELECT EntityNumber, NaceVersion
                               FROM activity
                               ''', connection)

df_Q4 = pd.DataFrame(sql_query_Q4)
# df_Q4["EntityNumber"]=df_Q4["EntityNumber"].drop_duplicates(keep:False)
df_Q4 = df_Q4.drop_duplicates(subset=["EntityNumber"], keep=False)
df_Q4 = df_Q4.groupby(["NaceVersion"])["EntityNumber"].count()
df_Q4


NaceVersion
2003     20804
2008    498296
Name: EntityNumber, dtype: int64

In [202]:
df_Q4M2008 = pd.read_csv("mapping_sector_2008.csv")
sql_query_Q4T1 = pd.read_sql_query ('''
                               SELECT EnterpriseNumber, StartDate, NaceCode
                               FROM enterprise
                               INNER JOIN activity ON enterprise.EnterpriseNumber = activity.EntityNumber
                               WHERE NaceVersion==2008
                               ''',connection)

df_Q4T1 = pd.DataFrame(sql_query_Q4T1)
df_Q4T1["NaceCode"] = df_Q4T1.NaceCode.str[:2].astype(int)
df_Q4T1["StartDate"] = df_Q4T1.StartDate.str[:10].replace('-','', regex=True)
#df['API'] = df['API'].str.replace('-','')
df_Q4T1

,EnterpriseNumber,StartDate,NaceCode
0,0200.065.765,19600809,84
1,0200.065.765,19600809,41
2,0200.068.636,19230216,36
3,0200.068.636,19230216,36
4,0200.068.636,19230216,93
...,...,...,...
2998239,0998.534.826,19980101,63
2998240,0998.549.771,18000101,35
2998241,0998.562.738,19990331,62
2998242,0998.576.495,20001231,82


# GRAPHIC DESIGN

### Q1 GRAPH

In [185]:
# Pie chart, where the slices will be ordered and plotted counter-clockwise:
fig1, ax1 = plt.subplots()
ax1.pie(df_Q1T1['Total_Enterprise'], labels = df_Q1T1['Description'], autopct='%1.1f%%')
st.pyplot(fig1)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

### Q2 GRAPH

In [183]:
#Display the number of active company
st.markdown(f"<h1 style='text-align: center; color: black;'>The number of active enterprise is :  </h1>", unsafe_allow_html=True)
st.markdown(f"<h1 style='text-align: center; color: grey;'>{score_active_ent} </h1>", unsafe_allow_html=True)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

### Q3 GRAPH

In [184]:
# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = df_Q3["Description"]
sizes = df_Q3["count(EnterpriseNumber)"]

fig3, ax3 = plt.subplots()
ax3.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax3.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

st.pyplot(fig3)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)